# This is a simple example script for 2D x/y stitching using muvis-align and the multiview-stitcher package

In [ ]:
import sys
sys.path.append('..')

from muvis_align.MVSRegistration import MVSRegistration
from muvis_align.util import dir_regex
from muvis_align.image.util import show_image

from multiview_stitcher import spatial_image_utils as si_utils
from multiview_stitcher import vis_utils

## Initialise registration parameters

In [ ]:
params = {
	'general':
	{
        'ui': 'mpl'
	},
	'operations':
	[
		{
			'operation': 'register',
			'input': '../data/S000/*.tiff',
			'source_metadata':
			{
				'scale': {'x': 0.032, 'y': 0.032},
				'position': {'y': 'fn[-3]*24', 'x': 'fn[-2]*24'}
			},
            'use_orthogonal_pairs': True,
			'transform_type': 'affine',
			'method':
			{
				'name': 'sift',
				'gaussian_sigma': 2,
				'inlier_threshold_factor': 0.05,
				'max_trials': 1000,
				'ransac_iterations': 3
			},
            'n_parallel_pairwise_regs': 1,
			'output': '../../output/'
		}
	]
}

## Get filenames

In [ ]:
operation_params = params['operations'][0]
filenames = dir_regex(operation_params['input'])
filenames

## Initialise muvis-align, initialise sims, and pre-process

In [ ]:
reg = MVSRegistration(params['general'])
reg.init_operation('', filenames, operation_params)
sims = reg.init_sims()
sims, norm_sims, _ = reg.preprocess(sims, operation_params)

for label, sim in zip(reg.file_labels, sims):
    print(label, si_utils.get_origin_from_sim(sim))

## Perform registration (using multiview-stitcher)

In [ ]:
%matplotlib inline
results = reg.register(sims, norm_sims)
registered_sims = results['sims']

## Show registration mapping

In [ ]:
mappings = results['mappings']
for label, mapping in zip(reg.file_labels, mappings.values()):
    print(f'{label}:\n', mapping.sel(t=0).data)

## Visualise registered sims

In [ ]:
%matplotlib inline
fig, ax = vis_utils.plot_positions(registered_sims, transform_key=reg.reg_transform_key, use_positional_colors=False, view_labels=reg.file_labels)

## Perform fusion (using multiview-stitcher)

In [ ]:
%matplotlib inline
fused = reg.fuse(registered_sims)

fused

## Output fused result

In [ ]:
%matplotlib inline
show_image(fused[0, 0])

In [ ]:
reg.save(reg.output + 'stitching2d', fused)